In [32]:
import requests
import json
import syft as sy
import pandas as pd

In [17]:
from enum import Enum
from syft.grid.grid_url import GridURL
from tqdm import tqdm

In [39]:
class EndPoints(Enum):
    METADATA = "api/v1/syft/metadata"
    STATUS = "api/v1/status"
    
class Status(Enum):
    OK = "Ok"
    FAILED = "Failed"

In [24]:
PUBLIC_NETWORKS_URL = "https://raw.githubusercontent.com/OpenMined/NetworkRegistry/main/networks.json"

def get_listed_public_networks(url=PUBLIC_NETWORKS_URL):
    response = requests.get(url, timeout=5)
    print(f"GET Public Network Status: {response.status_code}")
    response = response.json()
    network_list = response.get("networks", [])[1:]
    return network_list


def check_network_status(host_url):
    status = ""
    url = f"http://{host_url}/{EndPoints.STATUS.value}"
    response = requests.get(url, timeout=0.5)
    if response.status_code == 200:
        return Status.OK.value
    return Status.FAILED.value


def check_metadata_api(host_url):
    status = ""
    url = f"http://{host_url}/{EndPoints.METADATA.value}"
    response = requests.get(url, timeout=5)
    if response.status_code == 200:
        return Status.OK.value
    
    return Status.FAILED.value


def check_login_via_syft(host_url, retry=5):
    network_client = None
    grid_url = GridURL(host_or_ip=host_url)
    grid_url = grid_url.with_path("/api/v1")
    while(not network_client and retry > 0):
        
        try:
            network_client = sy.connect(url=grid_url, timeout=5)
        except TimeoutError:
            retry -= 1
    if network_client is None:
        return Status.FAILED.value
    
    print(network_client.name)
    return Status.OK.value


def check_ip_port(host_ip: str, port: int) -> bool:
    import socket
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(2)
        result = sock.connect_ex((host_ip, port))
        sock.close()
        if result == 0:
            return Status.OK.value
    except Exception:
        pass
    return Status.FAILED.value

In [48]:
import paramiko

def check_tailscale_status(host_ip: str, key_filename: str):
    ssh_client = paramiko.SSHClient()

    # To avoid an "unknown hosts" error. Solve this differently if you must...
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    # This mechanism uses a key_filename.    
    username = "azureuser"

    ssh_client.connect(hostname=host_ip, username=username, key_filename=key_filename)
    
    command = "TAILSCALE_CONTAINER_NAME=$(sudo docker ps --format '{{.Names}}' | grep -m 1 tailscale) &&"
    command += "sudo docker exec $TAILSCALE_CONTAINER_NAME tailscale status"
    
    stdin,stdout,stderr = ssh_client.exec_command(command)
    
    errors = stderr.readlines()
    
    if len(errors) > 0:
        return Status.FAILED.value
    
    stdout = stdout.readlines()
    if len(stdout) > 0:
        return Status.OK.value

    return Status.FAILED.value


def check_headscale_node_list(host_ip: str, key_filename: str):
    ssh_client = paramiko.SSHClient()

    # To avoid an "unknown hosts" error. Solve this differently if you must...
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    # This mechanism uses a key_filename.    
    username = "azureuser"

    ssh_client.connect(hostname=host_ip, username=username, key_filename=key_filename)
    
    command = "HEADSCALE_CONTAINER_NAME=$(sudo docker ps --format '{{.Names}}' | grep -m 1 headscale) && "
    command += "sudo docker exec $HEADSCALE_CONTAINER_NAME headscale nodes list -o json"
    
    stdin,stdout,stderr = ssh_client.exec_command(command)
    
    errors = stderr.readlines()
    print(errors)
    
    if len(errors) > 0:
        return Status.FAILED.value
    
    stdout = json.loads(stdout.read())
    if len(stdout) > 0:
        return stdout
    return Status.FAILED.value

In [41]:
network_list = get_listed_public_networks()

GET Public Network Status: 200


In [42]:
status_table_list = []

for network in tqdm(network_list):
    host_url = network["host_or_ip"]
    status = {}
    
    status["host_or_ip"] = host_url
    status["ssh_status"] = check_ip_port(host_url, port=80)
    status["ping_status"] = check_network_status(host_url)
    status["api_status"] = check_metadata_api(host_url)
    status["login_status"] = check_login_via_syft(host_url)
    status["tailscale_status"] = check_tailscale_status(host_url, key_filename="/home/shubham/PySyft/ssh_keys/networkMachine.pem")
    status["headscale_nodes_list"] = check_headscale_node_list(host_url, "/home/shubham/PySyft/ssh_keys/networkMachine.pem")
    
    status_table_list.append(status)

  0%|                                                                                                                                           | 0/10 [00:00<?, ?it/s]

govai_network_1


 10%|█████████████                                                                                                                      | 1/10 [00:08<01:12,  8.08s/it]

[]
govai_network_2


 20%|██████████████████████████▏                                                                                                        | 2/10 [00:15<01:03,  7.88s/it]

...

[]
govai_network_9


 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉             | 9/10 [01:10<00:07,  7.84s/it]

[]
govai_network_10


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:18<00:00,  7.82s/it]

[]


In [43]:
status_df = pd.DataFrame(status_table_list)

In [49]:
status_df

,host_or_ip,ssh_status,ping_status,api_status,login_status,tailscale_status,headscale_nodes_list
0,20.84.12.110,Ok,Ok,Ok,Ok,Ok,"b'[\n\t{\n\t\t""id"": 1,\n\t\t""machine_key"": ""40..."
1,20.84.14.55,Ok,Ok,Ok,Ok,Ok,"b'[\n\t{\n\t\t""id"": 1,\n\t\t""machine_key"": ""dd..."
2,20.84.13.24,Ok,Ok,Ok,Ok,Ok,"b'[\n\t{\n\t\t""id"": 1,\n\t\t""machine_key"": ""d6..."
3,20.84.14.84,Ok,Ok,Ok,Ok,Ok,"b'[\n\t{\n\t\t""id"": 1,\n\t\t""machine_key"": ""5b..."
4,20.84.12.64,Ok,Ok,Ok,Ok,Ok,"b'[\n\t{\n\t\t""id"": 1,\n\t\t""machine_key"": ""93..."
5,20.84.14.94,Ok,Ok,Ok,Ok,Ok,"b'[\n\t{\n\t\t""id"": 1,\n\t\t""machine_key"": ""71..."
6,20.84.12.138,Ok,Ok,Ok,Ok,Ok,"b'[\n\t{\n\t\t""id"": 1,\n\t\t""machine_key"": ""90..."
7,20.84.14.128,Ok,Ok,Ok,Ok,Ok,"b'[\n\t{\n\t\t""id"": 1,\n\t\t""machine_key"": ""1b..."
8,20.84.12.33,Ok,Ok,Ok,Ok,Ok,"b'[\n\t{\n\t\t""id"": 1,\n\t\t""machine_key"": ""94..."
9,20.84.8.208,Ok,Ok,Ok,Ok,Ok,"b'[\n\t{\n\t\t""id"": 1,\n\t\t""machine_key"": ""00..."


In [51]:
status_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>host_or_ip</th>\n      <th>ssh_status</th>\n      <th>ping_status</th>\n      <th>api_status</th>\n      <th>login_status</th>\n      <th>tailscale_status</th>\n      <th>headscale_nodes_list</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>20.84.12.110</td>\n      <td>Ok</td>\n      <td>Ok</td>\n      <td>Ok</td>\n      <td>Ok</td>\n      <td>Ok</td>\n      <td>b\'[\\n\\t{\\n\\t\\t"id": 1,\\n\\t\\t"machine_key": "40623f9de820d486101b34ed8d392485132864064962708900bc24dbd4aeaf1e",\\n\\t\\t"node_key": "bfb19632726c6590dd363077052400c3b3e3cf7d1e281cee6561b12cccfab66e",\\n\\t\\t"disco_key": "7fe19c71912b2e75348e1c20b71105c353e5f774980cbf5eca5a3186ffd4fb2f",\\n\\t\\t"ip_addresses": [\\n\\t\\t\\t"100.64.0.1"\\n\\t\\t],\\n\\t\\t"name": "govai-network-1",\\n\\t\\t"namespace": {\\n\\t\\t\\t"id": "1",\\n\\t\\t\\t"name": "omnet",\\n\\t\\t\\t"created_at": {

In [37]:
network_list[0]

{'name': 'GovAI Network 1',
 'host_or_ip': '20.84.12.110',
 'vpn_host_or_ip': '100.64.0.1',
 'protocol': 'http',
 'port': 80,
 'admin_email': 'support@openmined.org',
 'website': 'https://www.governance.ai/',
 'slack': 'https://slack.openmined.org/',
 'slack_channel': '#support'}

In [50]:
check_headscale_node_list(host_url, "/home/shubham/PySyft/ssh_keys/networkMachine.pem")

[]


[{'id': 1,
  'machine_key': '006e44a95f810669ce7a455e2eadd40e677e832e916b26e725eb23b1967bf368',
  'node_key': '77f0e0996426509ab4199851daf4adbde304dbc38bfb4ff56daf577ed81ff045',
  'disco_key': '1d9502da71ba6ffb2117062935fc87a783d8d1cf2fa1f579cc8d8e3c69054751',
  'ip_addresses': ['100.64.0.1'],
  'name': 'govai-network-10',
  'namespace': {'id': '1',
   'name': 'omnet',
   'created_at': {'seconds': 1654368873, 'nanos': 82248226}},
  'last_seen': {'seconds': 1654499601, 'nanos': 27277000},
  'last_successful_update': {'seconds': 1654499611, 'nanos': 28544971},
  'expiry': {'seconds': -62135596800},
  'pre_auth_key': {'namespace': 'omnet',
   'id': '1',
   'key': '94becb4e083e4532490e125214628c5dc28450cd7a588954',
   'used': True,
   'expiration': {'seconds': 1654372518, 'nanos': 598893378},
   'created_at': {'seconds': 1654368918, 'nanos': 604313287}},
  'created_at': {'seconds': 1654368920, 'nanos': 623426439}}]